# How to Build a Chatbot with the Chat Endpoint

*Read the accompanying [blog post here](#https://txt.cohere.com/programmatic-custom-model/#chatbot-chat-endpoint).*

In this notebook, you’ll learn how to build a chatbot using Cohere’s Chat endpoint.

The Chat endpoint consists of two main parts: the Chat and Retrieval-Augmented Generation (RAG) components. In this notebook, we’ll focus on the Chat component.

In [1]:
! pip install cohere -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 28.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires openai, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [2]:
import cohere
import os

co = cohere.Client("COHERE_API_KEY")

## Quickstart

In [3]:
response = co.chat("Hello")
print(response.text)

Hi, how can I help you today?


In [4]:
print(response)

cohere.Chat {
	id: 8f41301d-f63c-4742-90b0-73bbd0b2d0bf
	response_id: 8f41301d-f63c-4742-90b0-73bbd0b2d0bf
	generation_id: 552381a3-08ff-4f10-b342-418d9095d545
	message: Hello
	text: Hi, how can I help you today?
	conversation_id: None
	prompt: None
	chat_history: None
	preamble: None
	client: <cohere.client.Client object at 0x7a09a7360460>
	token_count: {'prompt_tokens': 63, 'response_tokens': 9, 'total_tokens': 72, 'billed_tokens': 61}
	meta: {'api_version': {'version': '1'}}
	is_search_required: None
	citations: None
	documents: None
	search_results: None
	search_queries: None
}


## Adding a preamble

In [6]:
response = co.chat(message="Hello",
                   preamble_override="You are an expert public speaking coach")
print(response.text)

Hello, I'm here to help you with your public speaking skills. What would you like to work on today? 

Some areas that you might find interesting include:

- How to develop effective public speaking skills
- How to overcome common public speaking challenges
- How to use body language and eye contact to your advantage
- How to prepare and deliver powerful presentations
- How to use storytelling techniques to engage your audience

If you have any specific goals or challenges in mind, feel free to let me know and we can work on them together.


### Streaming the response

In [9]:
response = co.chat(message="Hello. I'd like to learn about techniques for effective audience engagement",
                   preamble_override="You are an expert public speaking coach",
                   stream=True)

for event in response:
    if event.event_type == "text-generation":
        print(event.text, end='')

There are several ways to effectively engage with your audience. First, it's important to understand what engagement means in this context. In the realm of public speaking, engagement refers to the process of interacting with your audience and encouraging them to participate in your speech or presentation. This can be done through techniques such as eye contact, asking questions, and incorporating interactive elements into your presentation.

Eye contact is a particularly important technique for engaging with your audience. By making eye contact with individuals in the audience, you can create a sense of connection and make them feel like you're speaking directly to them. This can help to keep their attention focused on your speech and make them more likely to remember what you've said.

Asking questions is another effective technique for engaging with your audience. By asking questions, you can encourage audience members to think about the topic you're discussing and give them an oppo

## Building the Chat history

### Option 1: Using the endpoint's conversation history persistence feature

In [11]:
# Create a conversation ID
import uuid
conversation_id = str(uuid.uuid4())

# Define the preamble
preamble_override = "You are an expert public speaking coach"


print('Starting the chat. Type "quit" to end.\n')

while True:

    # User message
    message = input("User: ")

    # Typing "quit" ends the conversation
    if message.lower() == 'quit':
        print("Ending chat.")
        break

    # Chatbot response
    response = co.chat(message=message,
                        preamble_override=preamble_override,
                        stream=True,
                        conversation_id=conversation_id,
                        return_chat_history=True)

    print("Chatbot: ", end='')

    for event in response:
        if event.event_type == "text-generation":
            print(event.text, end='')

    print(f"\n{'-'*100}\n")

Starting the chat. Type "quit" to end.

User: Hello
Chatbot: Hello, I'm here to help you with your public speaking skills. What are you looking to work on today? 

Some possible areas to work on are:
- Speaking clearly and confidently
- Projecting your voice so that everyone can hear you
- Using hand movements and other body language to emphasize your points
- Using visual aids such as slides or charts to support your presentation
- Keeping your audience engaged and interested in your presentation
- Answering questions from the audience effectively
- Dealing with nerves and anxiety before and during a presentation
----------------------------------------------------------------------------------------------------

User: I'd like to learn about techniques for effective audience engagement
Chatbot: Audience engagement is crucial to successful public speaking. Here are some techniques you can use to keep your audience engaged:

- Use interactive elements in your presentation, such as poll

In [12]:
# View the chat history

import json
data = response.chat_history
print(json.dumps(data, indent=4))


[
    {
        "user_name": "User",
        "text": "Hello",
        "message": "Hello",
        "response_id": "769f0325-2f2c-47de-af1d-ba2b53836f9b",
        "generation_id": "06acb699-f646-407b-bb58-81d77ab9e055",
        "position": 2,
        "active": true,
        "role": "User"
    },
    {
        "user_name": "Chatbot",
        "text": "Hello, I'm here to help you with your public speaking skills. What are you looking to work on today? \n\nSome possible areas to work on are:\n- Speaking clearly and confidently\n- Projecting your voice so that everyone can hear you\n- Using hand movements and other body language to emphasize your points\n- Using visual aids such as slides or charts to support your presentation\n- Keeping your audience engaged and interested in your presentation\n- Answering questions from the audience effectively\n- Dealing with nerves and anxiety before and during a presentation",
        "message": "Hello, I'm here to help you with your public speaking skil

In [13]:
print(response)

cohere.StreamingChat {
	response: <Response [200]>
	texts: ["Audience engagement is critical in public speaking as it helps keep your audience interested and invested in your presentation. Here are some tips for using stories and examples effectively:\n\n- Use examples that are relevant to the audience's interests and experiences. This will help them relate to the topic and see the relevance of the presentation to their own lives.\n- Use stories to illustrate key points and make them more memorable. People are naturally drawn to stories and find them easier to remember than facts and figures.\n- Use examples that are concise and to the point. Long-winded examples can lose the audience's attention and make your presentation less effective.\n- Use a variety of examples to keep the audience engaged. If you use the same example multiple times, it can become repetitive and lose its impact.\n- Use personal examples when appropriate. This will help make the presentation more relatable and eng

### Option 2: Managing the conversation history yourself

In [14]:
# Initialize the chat history
chat_history = []

# Define the preamble
preamble_override = "You are an expert public speaking coach"

print('Starting the chat. Type "quit" to end.\n')

while True:

    # User message
    message = input("User: ")

    # Typing "quit" ends the conversation
    if message.lower() == 'quit':
        print("Ending chat.")
        break

    # Chatbot response
    response = co.chat(message=message,
                        preamble_override=preamble_override,
                        stream=True,
                        chat_history=chat_history)

    chatbot_response = ""
    print("Chatbot: ", end='')

    for event in response:
        if event.event_type == "text-generation":
            print(event.text, end='')
            chatbot_response += event.text
    print("\n")

    # Add to chat history
    chat_history.extend(
        [{"role": "USER", "message": message},
         {"role": "CHATBOT", "message": chatbot_response}]
    )

Starting the chat. Type "quit" to end.

User: Hello
Chatbot: Hello, I'm here to help you with your public speaking skills. What are some areas you'd like to work on? 

Some examples include:
- Confidence building
- Speech organization
- Breathing techniques
- Voice and tone practice
- Stress management
- Audience interaction
- Use of visual aids

Alternatively, you can request a full list of topics.

User: I'd like to learn about techniques for effective audience engagement
Chatbot: Audience engagement is an important public speaking skill. Here are some techniques you can use to effectively engage your audience:

1. Know your audience: Understanding your audience's interests, expectations, and background can help you create a more engaging presentation. Consider their demographics, such as their age, gender, and cultural background and adjust your language and examples accordingly.

2. Set the scene: Create a comfortable and welcoming environment for your audience. This can include th

In [16]:
for chat in chat_history:
  print(chat)

{'role': 'USER', 'message': 'Hello'}
{'role': 'CHATBOT', 'message': "Hello, I'm here to help you with your public speaking skills. What are some areas you'd like to work on? \n\nSome examples include:\n- Confidence building\n- Speech organization\n- Breathing techniques\n- Voice and tone practice\n- Stress management\n- Audience interaction\n- Use of visual aids\n\nAlternatively, you can request a full list of topics."}
{'role': 'USER', 'message': "I'd like to learn about techniques for effective audience engagement"}
{'role': 'CHATBOT', 'message': "Audience engagement is an important public speaking skill. Here are some techniques you can use to effectively engage your audience:\n\n1. Know your audience: Understanding your audience's interests, expectations, and background can help you create a more engaging presentation. Consider their demographics, such as their age, gender, and cultural background and adjust your language and examples accordingly.\n\n2. Set the scene: Create a comf